In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import random
import os

gpus = tf.config.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

keras.mixed_precision.set_global_policy('mixed_float16')

tf.config.optimizer.set_jit(True)

os.environ['TF_GPU_THREAD_MODE'] = 'gpu_private'

In [ ]:
training_path = "./data/training"
label_to_int = {'cycling': '0', 'jogging': '1', 'sitting': '2'}
training_list = []

for label in os.listdir(training_path):
    image_path = os.path.join(training_path, label)
    for filename in os.listdir(image_path):
        training_list.append((os.path.join(image_path, filename), label_to_int[label]))

random.shuffle(training_list)
training_list = tf.data.Dataset.from_tensor_slices(training_list)
print("Number of training data:", len(training_list))

In [ ]:
def preprocess_data(sample):
    
    image = tf.io.read_file(sample[0])
    image = tf.io.decode_image(image, channels=3, expand_animations=False)
    image = tf.image.resize(image, [71, 71], antialias=True)
    
    label = sample[1]
    label = tf.strings.to_number(label, out_type='int32')
    
    return image, label

In [ ]:
def data_augmentation(image, label):
    
    image = tf.image.random_brightness(image, 64)
    image = tf.image.random_hue(image, 0.1)
    image = tf.clip_by_value(image, 0., 255.)
    image = tf.image.random_flip_left_right(image)
    
    return image, label

In [ ]:
BATCH_SIZE = 48
training_data = training_list.map(
    preprocess_data,
    num_parallel_calls=tf.data.AUTOTUNE).batch(BATCH_SIZE, drop_remainder=True).cache().prefetch(tf.data.AUTOTUNE)

In [ ]:
pre_trained_model = keras.applications.MobileNetV3Small(
    input_shape=(71, 71, 3), 
    include_top=False,
    pooling='avg',
    weights='imagenet')

for layer in pre_trained_model.layers:
    layer.trainable = False

NAME = 'classification_layer'
x = layers.Dense(3, dtype='float32', name="classification")(pre_trained_model.output)
model = keras.Model(pre_trained_model.input, x, name=NAME)

model.compile(
    optimizer='adam',
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'])

model.summary()

In [ ]:
EPOCHS = 200

history = model.fit(
    training_data,
    epochs=EPOCHS,
    verbose=0)

In [ ]:
plt.plot(history.history['loss'], label='training loss')
plt.title('Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid()
plt.show()

plt.plot(history.history['accuracy'], label='training accuracy')
plt.title('Training Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.grid()
plt.show()